In [1]:
import tensorflow as tf
import numpy as np
import sys

from sklearn.metrics import mean_squared_error
from keras.callbacks import LearningRateScheduler
#from sklearn.preprocessing import MinMaxScaler ##########

#sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
sys.path.append('/Users/Think/Desktop/project')

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression

#Data handlers
from ann_framework.data_handlers.data_handler_Oscillator import OscillatorDataHandler

#Custom modules
from ann_framework import aux_functions

import aux_functions_stochastic
import analytic_functions
import loss_functions


D:\Anaconda3\Anaconda3_Python6\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#global variables

deltas = [0.1, 0.1]

k = 1
c = 0.1
D = 1
num_fevals = 5

sigma_x = np.sqrt(D / (k * c))
sigma_y = np.sqrt(D / c)

### Define tensorflow model

In [14]:
def create_placeholders(input_shape, output_shape):
    
    X = tf.placeholder(tf.float32, shape=(None,input_shape), name="X")
    y = tf.placeholder(tf.float32, shape=None, name="y")
    
    return X, y

"""
def tf_model(X):
    
    l2_lambda_regularization = 0.20
    l1_lambda_regularization = 0.10
    
    A1 = tf.layers.dense(X, 20, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), 
                         name="fc1")
    A2 = tf.layers.dense(A1, 20, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc2")
    y = tf.layers.dense(A2, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="out")
    
    return y
"""

def tf_model_yulin(X):
    
    A1 = tf.layers.dense(X, 3000, activation=tf.nn.tanh, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),name="fc1")
    A2 = tf.layers.dense(A1, 3000, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),name="fc2")
    """
    A3 = tf.layers.dense(A2, 20, activation=tf.nn.elu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc3")
    dropout3 = tf.layers.dropout(inputs = A3, rate = 0.2)
    A4 = tf.layers.dense(dropout3, 20, activation=tf.nn.elu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc4")
    dropout4 = tf.layers.dropout(inputs = A4, rate = 0.2)
    """
    y = tf.layers.dense(A2, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        name="out")
    #activation funcs
    
    return y


### Create Model TF

In [15]:
def tf_compiled_model(num_features, output_shape):
    tf.reset_default_graph()

    X, y = create_placeholders(num_features, output_shape)

    #y_pred = tf_model(X)
    y_pred = tf_model_yulin(X)
    
    loss_function = loss_functions.squared_residual_function_wrapper2(k, c, D, deltas, num_fevals, sigma_x, sigma_y)
    cost, e = loss_function(X, y_pred, y)
    #reg_cost = tf.losses.get_regularization_loss()
    total_cost = e
    R = cost / (deltas[0]*deltas[1])
    
    # total_cost: e
    # cost: R
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01, beta1=0.5).minimize(R)

    return {'X_placeholder': X, 'y_placeholder': y, 'y_pred': y_pred, 'cost': R, 'total_cost': total_cost, 'optimizer': optimizer}

### Create Tunable Model Tensorflow and assign data

In [16]:
#scaler = MinMaxScaler()

dhandler_stochastic = OscillatorDataHandler()

#(data_scaler = scaler)

model = tf_compiled_model(2, 1)

tModel = SequenceTunableModelRegression('ModelStochastic_SN_1', model, lib_type='tensorflow', 
                                        data_handler=dhandler_stochastic)

tModel.load_data(verbose=1, cross_validation_ratio=0.2, x=[0, 0], boundaries=[10, 10], n=[50, 50])

#Real phi function
p_real_test, phi_real_test = analytic_functions.real_p(tModel.X_test[:,0], tModel.X_test[:,1], sigma_x, sigma_y)
p_real_train, phi_real_train = analytic_functions.real_p(tModel.X_train[:,0], tModel.X_train[:,1], sigma_x, sigma_y)
p_real_crossVal, phi_real_crossVal = analytic_functions.real_p(tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], sigma_x, sigma_y)

tModel.y_test = phi_real_test
tModel.y_train = phi_real_train
tModel.y_crossVal = phi_real_crossVal


tModel.print_data()

Loading data. Cros-Validation ratio 0.2
Printing shapes

Training data (X, y)
(7200, 2)
(7200, 1)
Cross-Validation data (X, y)
(1800, 2)
(1800, 1)
Testing data (X, y)
(1000, 2)
(1000, 1)
Printing first 5 elements

Training data (X, y)
[[-6.16161616 -9.7979798 ]
 [ 9.5959596   5.15151515]
 [ 6.56565657 -6.16161616]
 [ 1.71717172  3.33333333]
 [-0.3030303  -9.5959596 ]]
[[6.69829609]
 [5.93102745]
 [4.05366799]
 [0.70298949]
 [4.6087134 ]]
Cross-Validation data (X, y)
[[-2.92929293 -9.7979798 ]
 [ 7.17171717  6.56565657]
 [-2.72727273 -5.95959596]
 [ 8.38383838 -9.7979798 ]
 [-5.35353535 -2.12121212]]
[[5.22905826]
 [4.72706867]
 [2.14774003]
 [8.31445771]
 [1.65799408]]
Testing data (X, y)
[[-8.38383838  2.52525253]
 [ 1.11111111  4.34343434]
 [ 9.19191919 -7.37373737]
 [-1.91919192 -6.16161616]
 [-7.17171717  6.96969697]]
[[3.83328232]
 [1.00499949]
 [6.94316906]
 [2.08244057]
 [5.00051015]]


D:\Anaconda3\Anaconda3_Python6\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Train model Tensorflow

In [ ]:
tModel.epochs = 100
#lrate = LearningRateScheduler(aux_functions.step_decay)
minibatches_function_handle = aux_functions_stochastic.get_minibatches

#tf.get_variable("deltas", initializer=deltas)

sess = tf.Session()

#writer = tf.summary.FileWriter('./graphs', sess.graph)

tModel.train_model(tf_session=sess, get_minibatches_function_handle=minibatches_function_handle, 
                   verbose=1, deltas=deltas)

    # cost_reg: total_cost -> e
    # cost: R

Epoch: 0001 cost_reg= 208.646775850 cost= 150.295520318
Epoch: 0002 cost_reg= 4.153647598 cost= 0.018471716
Epoch: 0003 cost_reg= 3.993827804 cost= 0.007236687
Epoch: 0004 cost_reg= 3.951905886 cost= 0.005198312
Epoch: 0005 cost_reg= 3.933375502 cost= 0.004259488
Epoch: 0006 cost_reg= 4.023547633 cost= 0.004464180
Epoch: 0007 cost_reg= 4.019555902 cost= 0.004816544
Epoch: 0008 cost_reg= 4.122024409 cost= 0.012224358
Epoch: 0009 cost_reg= 4.557755534 cost= 0.063423628
Epoch: 0010 cost_reg= 5.325601101 cost= 0.100315926
Epoch: 0011 cost_reg= 5.687167152 cost= 0.032884387
Epoch: 0012 cost_reg= 5.674229558 cost= 0.007767939
Epoch: 0013 cost_reg= 6.978184446 cost= 0.241748166
Epoch: 0014 cost_reg= 9.034848785 cost= 0.347220451
Epoch: 0015 cost_reg= 8.911173820 cost= 0.002598947
Epoch: 0016 cost_reg= 8.966331673 cost= 0.001568223


In [ ]:
display_points = 50

"""
#Evaluate real model
X_test = tModel.X_crossVal

#Real phi function
sigma_x = np.sqrt(D / (k * c))
sigma_y = np.sqrt(D / c)
p_real, phi_real = analytic_functions.real_p(X_test[:,0], X_test[:,1], sigma_x, sigma_y)

tModel.y_crossVal = phi_real

#p_real = np.ravel(p_real)
#phi_real = np.ravel(phi_real)

tModel.evaluate_model(['mse', 'rmse'], cross_validation=True, tf_session=sess)
#phi_pred = np.ravel(tModel.y_predicted)
phi_pred = tModel.y_predicted

d = 2 * np.pi * sigma_x * sigma_y
c_not = 1/d
p_pred = c_not * np.exp(-phi_pred)
"""

tModel.evaluate_model(['mse', 'rmse'], cross_validation=True, tf_session=sess)
X_test = tModel.X_crossVal
phi_pred = tModel.y_predicted
phi_real = tModel.y_crossVal
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
mse = cScores['mse']
time = tModel.train_time

total_points = len(phi_pred)
sample_array = list(range(total_points))

sample_points = np.random.choice(sample_array, display_points)

phi_real_sampled = phi_real[sample_points]
phi_pred_sampled = phi_pred[sample_points]
X_sampled = X_test[sample_points,:]

i = range(len(phi_pred_sampled))


for i, phi_real_display, phi_pred_display in zip(i, phi_real, phi_pred):
    print('xy {}, Real Phi {}, Predicted Phi {}'.format(X_test[i], phi_real_display, phi_pred_display))

#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("MSE: {}".format(mse))
print("Time : {} seconds".format(time))

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig1 = plt.figure(1)
ax1 = Axes3D(fig1)
pred_phi_plot = Axes3D.scatter(ax1, tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], phi_pred)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('phi(x, y)_pred')

plt.show()

In [ ]:
fig2 = plt.figure(2)
ax2 = Axes3D(fig2)
real_phi_plot = Axes3D.scatter(ax2, tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], phi_real)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('phi(x, y)_real')

plt.show()

In [ ]:
fig3 = plt.figure(3)
ax3 = Axes3D(fig3)
pred_p_plot = Axes3D.scatter(ax3, tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], p_pred)
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_zlabel('p(x, y)_pred')

plt.show()

In [ ]:
fig4 = plt.figure(4)
ax4 = Axes3D(fig4)
real_p_plot = Axes3D.scatter(ax4, tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], p_real)
ax4.set_xlabel('x')
ax4.set_ylabel('y')
ax4.set_zlabel('p(x, y)_real')

plt.show()

In [ ]:
def real_derivatives(X, sigma_x, sigma_y):
    x1 = X[:, 0]
    x2 = X[:, 1]
    
    first_order_dx = x1/(sigma_x**2)
    first_order_dy = x2/(sigma_y**2)
    second_order_dy = 1/(sigma_y**2)
    
    return first_order_dx, first_order_dy, second_order_dy

dx, dy, ddy = real_derivatives(X_test, sigma_x, sigma_y)

In [ ]:
dx, dy, ddy = analytic_functions.real_derivatives(tModel.X_crossVal, sigma_x, sigma_y)

#for i in range(len(dx)):
 #   print('dx {}, dy {}, ddy {}'.format(dx[i], dy[i], ddy[i]))